In [47]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [48]:
def remove_punctuations(text):
    for punctuation in string.punctuation:  # Fixed typo here
        text = text.replace(punctuation, '')
    return text

In [49]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [50]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [51]:
vocab =  pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [52]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [53]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in x.split()))
    data["tweet"] = data['tweet'].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace(r'\d+', '', regex=True)
    data["tweet"] =  data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] =  data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [58]:
def vectorizer(ds, vocabulary):
    vectorized_1st = []  # Initialize the list before using it

    for sentence in ds:
        sentence_1st = [0] * len(vocabulary)
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_1st[i] = 1  # Use 'sentence_1st' instead of 'sentence_list'
        
        vectorized_1st.append(sentence_1st)  # Append sentence_1st, not sentence_list

    vectorized_1st_new = np.asarray(vectorized_1st, dtype=np.float32)

    return vectorized_1st_new


In [59]:
def get_prediction(vectorized_text):
    predection = model.predict(vectorized_text)
    if predection == 1:
        return 'negative'
    else:
        return 'positive'

In [65]:
txt = "awesome"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'